# DATSCI W261 ASSIGNMENT 3
Section 2<br/>
Student: Kuan Lin<br/>
Date: 1/29/2016

* <b>HW3.0. What is a merge sort? Where is it used in Hadoop? How is  a combiner function in the context of Hadoop? Give an example where it can be used and justify why it should be used in the context of this problem. What is the Hadoop shuffle?</b>

to do...

* <b>HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress). Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc.</b>

In [1]:
%%writefile mapper.py
#!/usr/bin/python
# mapper to emit counts for the three defined categories
import sys
for line in sys.stdin:    
    line = line.strip().lower()
    if line == '' or line.startswith('complaint id'): continue
        
    # get the complaint category
    category = line.split(',')[1].strip()
    counter_name = 'other_complaints'
    if category == 'debt collection':
        counter_name = 'debt_collection_complaints'
    elif category == 'mortgage':
        counter_name = 'mortgage_complaints'
    
    # update hadoop counters:
    sys.stderr.write("reporter:counter:customer_complaints,%s,1\n"%counter_name)
    # emit regular mapper result
    print '%s\t%s' % (counter_name, 1)

Writing mapper.py


In [11]:
%%writefile reducer.py
#!/usr/bin/python

import sys

current_category = None
current_count = 0
category = None
for line in sys.stdin:
    category, count = line.strip().split('\t', 1)
    if category == current_category:
        current_count += int(count)
    else:
        if current_category != None:
            # write result to STDOUT
            print '%s\t%s' % (current_category, current_count)
        current_category = category
        current_count = int(count)
if category == current_category:
    print '%s\t%s' % (current_category, current_count)

Overwriting reducer.py


In [12]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_1_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -mapper mapper.py -file /data/w261/hw3/mapper.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/Consumer_Complaints.csv -output /w261/hw3/hw3_1_output;

16/01/30 01:24:08 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_1_output
16/01/30 01:24:10 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob5504775449930135467.jar tmpDir=null
16/01/30 01:24:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 01:24:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 01:24:18 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 01:24:18 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 01:24:19 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454112721658_0005
16/01/30 01:24:19 INFO impl.YarnClientImpl: Submitted application application_1454112721658_0005
16/01/30 01:24:20 INFO mapreduce.Job: 

In [13]:
# examing the results
!hdfs dfs -cat /w261/hw3/hw3_1_output/part-00000

debt_collection_complaints	44372
mortgage_complaints	125752
other_complaints	142788


* <b>HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters</b>
Perform a word count analysis of this single record dataset using a Mapper and Reducer 
based WordCount (i.e., no combiners are used here) using user defined Counters to count up 
how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter,
 and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively.
 Please explain.
 Please use multiple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using 
a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined 
Counters to count up how many time the mapper and reducer are called. What is the value of 
your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, 
Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined 
Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your 
user defined Mapper Counter, and Reducer Counter after completing your word count job. 
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? 
Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and 
their frequency and their relative frequency. If there are ties please sort the tokens in 
alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [15]:
%%writefile hw3_2_input.txt
foo foo quux labs foo bar quux

Writing hw3_2_input.txt


In [16]:
# put the input file into hdfs
!hdfs dfs -put hw3_2_input.txt /w261/hw3

In [22]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")
for line in sys.stdin:    
    line = line.strip().lower()
    if line == '': continue
    for word in line.split(' '):
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


In [38]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# record reducer invokation
sys.stderr.write("reporter:counter:custom_counters,reducer_count,1\n")
from operator import itemgetter

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    if len(line.split('\t')) != 2: continue

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


In [28]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_2_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -D mapred.map.tasks=1 -D mapred.reduce.tasks=4 -mapper mapper.py -file /data/w261/hw3/mapper.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/hw3_2_input.txt -output /w261/hw3/hw3_2_output

rm: `/w261/hw3/hw3_2_output': No such file or directory
16/01/30 02:07:58 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob1960028508392768622.jar tmpDir=null
16/01/30 02:08:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:08:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:08:06 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:08:06 INFO mapreduce.JobSubmitter: number of splits:1
16/01/30 02:08:06 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/30 02:08:06 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
16/01/30 02:08:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454112721658_0008
16/01/30 02:0

In [32]:
!hdfs dfs -cat /w261/hw3/hw3_2_output/part-0000*

quux	2
foo	3
bar	1
labs	1


Since there is only one line and four distinct words, the optimal setting should be using 1 mapper and 4 reducers.  However, it seems that the default setting for Hadoop is to do 2 maper and 1 reducer.  If I specify mapper and reducer task counts in Hadoop Streaming parameter and 1 and 4, I now get 1 and 4 as specified in the homework problem.

* <b>HW3.2 continues</b>: Please use multiple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using 
a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined 
Counters to count up how many time the mapper and reducer are called. What is the value of 
your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [36]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")
for line in sys.stdin:    
    line = line.strip().lower()
    if line == '' or line.startswith("complaint id"): continue
    issue_text = line.split(',')[3]
    print '%s\t%s' % (issue_text, 1)

Overwriting mapper.py


In [39]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_2_1_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -D mapred.map.tasks=2 -D mapred.reduce.tasks=2 -mapper mapper.py -file /data/w261/hw3/mapper.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/Consumer_Complaints.csv -output /w261/hw3/hw3_2_1_output

16/01/30 02:28:36 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_2_1_output
16/01/30 02:28:38 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob1057362170451352704.jar tmpDir=null
16/01/30 02:28:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:28:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:28:46 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:28:46 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 02:28:46 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/30 02:28:46 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.m

The mapper and reducer counter are both 2, which is exactly what I specified in the parameters.  This is not always going to be the case though - we don't have direct control over how Hadoop is goint to split jobs.  The parameters is just a hint for the system.

* <b>HW3.2 continues</b>Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, 
Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined 
Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your 
user defined Mapper Counter, and Reducer Counter after completing your word count job.

In [35]:
%%writefile combiner.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,combiner_count,1\n")
from operator import itemgetter

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    if len(line.split('\t')) != 2: continue

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting combiner.py


In [36]:
!chmod +x mapper.py; chmod +x reducer.py; chmod +x combiner.py
!hdfs dfs -rm -r /w261/hw3/hw3_2_2_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -D mapred.map.tasks=2 -D mapred.reduce.tasks=2 -mapper mapper.py -file /data/w261/hw3/mapper.py -combiner combiner.py -file /data/w261/hw3/combiner.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/Consumer_Complaints.csv -output /w261/hw3/hw3_2_2_output
#!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar -D mapred.map.tasks=2 -D mapred.reduce.tasks=2 -mapper mapper.py -file /data/w261/hw3/mapper.py -reducer reducer.py -file /data/w261/hw3/reducer.py -input /w261/hw3/Consumer_Complaints.csv -output /w261/hw3/hw3_2_2_output

16/01/30 04:11:29 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_2_2_output
16/01/30 04:11:31 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/combiner.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob7026719206050507258.jar tmpDir=null
16/01/30 04:11:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 04:11:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 04:11:38 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 04:11:38 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 04:11:38 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/30 04:11:38 INFO Configuration.deprecation: mapred.map.tasks is deprecated. 

Combiner Counter: 4<br/>
Mapper Counter: 2<br/>
Reducer Counter: 2<br/>

* <b>HW3.2 Continue</b>: Using a single reducer: What are the top 50 most frequent terms in your word count analysis? 
Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in 
alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [37]:
!hdfs dfs -cat /w261/hw3/hw3_2_2_output/* | head -10

"account opening	16205
"making/receiving payments	3226
advertising and marketing	1193
arbitration	168
balance transfer	502
balance transfer fee	95
bankruptcy	222
can't contact lender	221
cash advance	136
cash advance fee	104
cat: Unable to write to output stream.


In [21]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import csv
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")

total_count = 0
# input comes from STDIN
for line in sys.stdin:
    line = line.strip().lower()
    if line == '' or line.startswith("complaint id"): continue
    lineArr = list(csv.reader(line.splitlines(), delimiter=',', quotechar='"'))[0]
    issue_text = lineArr[3]
    total_count += 1
    if issue_text.strip() == '': issue_text = 'blank'
    print '%s\t%s\t%s' % ("issue_words", issue_text, 1)

# print total word count from this mapper to calculate relative frequency
print '%s\t%s\t%s' % ("issue_words", "*", total_count)

Overwriting mapper.py


In [25]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# record reducer invokation
sys.stderr.write("reporter:counter:custom_counters,reducer_count,1\n")
total_count = 0
current_issue = None
current_count = 0
for line in sys.stdin:
    line = line.strip()
    if line == '': continue
    lineArr = line.split('\t')
    issue_text = lineArr[1]
    issue_count = int(lineArr[2])
    # aggregate total counts
    if issue_text == "*":
        total_count += issue_count
    else:
        if issue_text == current_issue:
            current_count += issue_count
        else:
            if current_issue != None and total_count != 0:
                print "%s\t%s\t%.6f" % (current_issue, current_count, float(current_count)/float(total_count))
            current_issue = issue_text
            current_count = issue_count
if current_issue != None and total_count != 0:
    print "%s\t%s\t%.6f" % (current_issue, current_count, float(current_count)/float(total_count))

Overwriting reducer.py


In [26]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_2_3_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar \
-D stream.num.map.output.key.fields=3 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k3nr -k2" \
-mapper mapper.py -file /data/w261/hw3/mapper.py \
-reducer reducer.py -file /data/w261/hw3/reducer.py \
-input /w261/hw3/Consumer_Complaints.csv \
-output /w261/hw3/hw3_2_3_output;

16/01/30 22:35:59 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_2_3_output
16/01/30 22:36:02 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob3516562719083472012.jar tmpDir=null
16/01/30 22:36:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 22:36:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 22:36:09 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 22:36:09 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 22:36:09 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/01/30 22:36:09 INFO Configuration.deprecation: mapred.text.key.comparato

Top 50 frequent items and their relative frequencies:

In [28]:
!hdfs dfs -cat /w261/hw3/hw3_2_3_output/part-00000 | sort -t$'\t' -k2,2nr -k1,1 | head -50

loan modification,collection,foreclosure	70487	0.225261
loan servicing, payments, escrow account	36767	0.117499
incorrect information on credit report	29069	0.092898
cont'd attempts collect debt not owed	17972	0.057435
account opening, closing, or management	16205	0.051788
deposits and withdrawals	10555	0.033732
communication tactics	8671	0.027711
application, originator, mortgage broker	8625	0.027564
disclosure verification of debt	7655	0.024464
billing disputes	6938	0.022172
other	6273	0.020047
problems caused by my funds being low	5663	0.018098
credit reporting company's investigation	4858	0.015525
managing the loan or lease	4560	0.014573
unable to get credit report/credit score	4357	0.013924
settlement process and costs	4350	0.013902
repaying your loan	3844	0.012285
problems when you are unable to pay	3821	0.012211
false statements or representation	3621	0.011572
improper contact or sharing of info	3489	0.011150
apr or interest rate	3431	0.010965
identity theft / fraud / embezzleme

Bottom 10 tokens and their counts and relative frequencies

In [29]:
!hdfs dfs -cat /w261/hw3/hw3_2_3_output/part-00000 | sort -t$'\t' -k2,2n -k1,1 | head -10

blank	4	0.000013
incorrect/missing disclosures or info	64	0.000205
charged bank acct wrong day or amt	71	0.000227
convenience checks	75	0.000240
payment to acct not credited	92	0.000294
balance transfer fee	95	0.000304
wrong amount charged or received	98	0.000313
cash advance fee	104	0.000332
received a loan i didn't apply for	118	0.000377
overlimit fee	127	0.000406


* <b>HW3.3. Shopping Cart Analysis</b><br/>
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.<br/>
Do some exploratory data analysis of this dataset. 
How many unique items are available from this supplier?
Using a single reducer: Report your findings such as number of unique products; 
largest basket; report the top 50 most frequently purchased items,  their frequency,  
and their relative frequency (break ties by sorting the products alphabetical order) etc. 
using Hadoop Map-Reduce. 

In [38]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")

total_count = 0
largest_basket_size = 0
# input comes from STDIN
for line in sys.stdin:
    line = line.strip().lower()
    if line == '': continue
    lineArr = line.split(' ')
    # record largest basket per mapper
    if len(lineArr) > largest_basket_size:
        largest_basket_size = len(lineArr)
    for product in lineArr:
        total_count += 1
        print '%s\t%s\t%s' % ("products", product, 1)

# print total word count from this mapper to calculate relative frequency
print '%s\t%s\t%s' % ("products", "*", total_count)
# print additional meta data for EDA analysis
print '%s\t%s\t%s' % ("meta_data", "largest_basket", largest_basket_size)

Overwriting mapper.py


In [41]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# record reducer invokation
sys.stderr.write("reporter:counter:custom_counters,reducer_count,1\n")
total_count = 0
current_issue = None
current_count = 0
largest_basket_size = 0
unique_products = set()
for line in sys.stdin:
    line = line.strip()
    if line == '': continue
    lineArr = line.split('\t')
    issue_text = lineArr[1]
    issue_count = int(lineArr[2])
    # record largest basket
    if issue_text == 'largest_basket':
        if issue_count > largest_basket_size:
            largest_basket_size = issue_count
        continue
    
    # aggregate total counts
    if issue_text == "*":
        total_count += issue_count
    else:
        unique_products.add(issue_text)
        if issue_text == current_issue:
            current_count += issue_count
        else:
            if current_issue != None and total_count != 0:
                print "%s\t%s\t%.6f" % (current_issue, current_count, float(current_count)/float(total_count))
            current_issue = issue_text
            current_count = issue_count
if current_issue != None and total_count != 0:
    print "%s\t%s\t%.6f" % (current_issue, current_count, float(current_count)/float(total_count))
print "----meta data below-----"
print "Total Unique Products: %s" % len(unique_products)
print "Largest Basket Size: %s" % largest_basket_size

Overwriting reducer.py


In [42]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_3_1_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar \
-D stream.num.map.output.key.fields=3 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k3nr -k2" \
-mapper mapper.py -file /data/w261/hw3/mapper.py \
-reducer reducer.py -file /data/w261/hw3/reducer.py \
-input /w261/hw3/ProductPurchaseData.txt \
-output /w261/hw3/hw3_3_1_output;

16/01/30 23:55:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_3_1_output
16/01/30 23:55:27 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob2104852217751772906.jar tmpDir=null
16/01/30 23:55:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 23:55:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 23:55:34 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 23:55:35 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 23:55:35 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/01/30 23:55:35 INFO Configuration.deprecation: mapred.text.key.comparato

In [43]:
# total unique products and largest baskets
!hdfs dfs -cat /w261/hw3/hw3_3_1_output/part* | tail -3

----meta data below-----	
Total Unique Products: 12592	
Largest Basket Size: 37	


In [44]:
# top 50 products, their frequencies and relative frequencies
!hdfs dfs -cat /w261/hw3/hw3_3_1_output/part* | sort -t$'\t' -k2,2nr -k1,1 | head -50

dai62779	6667	0.017507
fro40251	3881	0.010191
ele17451	3875	0.010175
gro73461	3602	0.009458
sna80324	3044	0.007993
ele32164	2851	0.007486
dai75645	2736	0.007184
sna45677	2455	0.006447
fro31317	2330	0.006118
dai85309	2293	0.006021
ele26917	2292	0.006019
fro80039	2233	0.005864
gro21487	2115	0.005554
sna99873	2083	0.005470
gro59710	2004	0.005262
gro71621	1920	0.005042
fro85978	1918	0.005036
gro30386	1840	0.004832
ele74009	1816	0.004769
gro56726	1784	0.004685
dai63921	1773	0.004656
gro46854	1756	0.004611
ele66600	1713	0.004498
dai83733	1712	0.004496
fro32293	1702	0.004469
ele66810	1697	0.004456
sna55762	1646	0.004322
dai22177	1627	0.004272
fro78087	1531	0.004020
ele99737	1516	0.003981
ele34057	1489	0.003910
gro94758	1489	0.003910
fro35904	1436	0.003771
fro53271	1420	0.003729
sna93860	1407	0.003695
sna90094	1390	0.003650
gro38814	1352	0.003550
ele56788	1345	0.003532
gro61133	1321	0.003469
dai88807	1316	0.003456
ele74482	1316	0.003456
ele59935	1311	0.003443
sna96271	1295	0.003401
dai43223	12

* <b>HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs</b>
<p>Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. 
Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) 
that have a support count of 100 or more.</p>
<p>List the top 50 product pairs with corresponding support count (aka frequency), 
and relative frequency or support (number of records where they coccur, 
the number of records where they coccur/the number of baskets in the dataset)  
in decreasing order of support  for frequent (100>count) itemsets of size 2.</p>
<p>Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. 
Free free to use combiners if they bring value. 
Instrument your code with counters for count the number of times your mapper, 
combiner and reducers are called.</p>
<p>Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 
<br/>
Report  the compute time for the Pairs job. Describe the computational setup used 
(E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called 
using Counters and report these counts.</p>


In [23]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")
total_baskets = 0
for line in sys.stdin:
    line = line.strip().upper()
    if line == '': continue
    total_baskets += 1
    lineArr = sorted(line.split(' '))
    for i, item in enumerate(lineArr):
        j = i + 1
        while j < len(lineArr):
            print "%s\t%s\t%s\t%s"%("paired_strategy", item, lineArr[j], 1)
            j += 1
print "%s\t%s\t%s\t%s"%("paired_strategy", "*", "*", total_baskets)

Overwriting mapper.py


In [24]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# record reducer invokation
sys.stderr.write("reporter:counter:custom_counters,reducer_count,1\n")

total_baskets = 0
support_threshold = 100

current_pair = None
current_support = 0

for line in sys.stdin:
    line = line.strip().upper()
    if line == '': continue
    lineArr = line.split('\t')
    item1 = lineArr[1]
    item2 = lineArr[2]
    pair = (item1, item2)
    pair_count = int(lineArr[3])
    # gather total basket count first
    if item1 == "*" and item2 == "*":
        total_baskets += pair_count
        continue
    if current_pair == pair:
        current_support += pair_count
    else:
        if current_pair != None and current_support > support_threshold:
            print "%s\t%s\t%s\t%.6f" % (current_pair[0], current_pair[1], current_support, float(current_support)/float(total_baskets))
        current_pair = pair
        current_support = pair_count
        
if current_pair != None and current_support > support_threshold:
    print "%s\t%s\t%s\t%.6f" % (current_pair[0], current_pair[1], current_support, float(current_support)/float(total_baskets))

Overwriting reducer.py


In [25]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_4_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar \
-D stream.num.map.output.key.fields=3 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k2 -k3" \
-mapper mapper.py -file /data/w261/hw3/mapper.py \
-reducer reducer.py -file /data/w261/hw3/reducer.py \
-input /w261/hw3/ProductPurchaseData.txt \
-output /w261/hw3/hw3_4_output;

16/01/31 02:20:28 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw3/hw3_4_output
16/01/31 02:20:30 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob1768631432912676945.jar tmpDir=null
16/01/31 02:20:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 02:20:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 02:20:38 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 02:20:38 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 02:20:38 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/01/31 02:20:38 INFO Configuration.deprecation: mapred.text.key.comparator.

In [26]:
# top 50 most co-occuring pairs.  break tie by taking the first in lexicographically increasing order
import subprocess
cat = subprocess.Popen(["hadoop", "fs", "-cat", "/w261/hw3/hw3_4_output/part*"], stdout=subprocess.PIPE)
all_pairs = []
for line in cat.stdout:
    all_pairs.append(line.strip().split('\t'))
# first sort by support, then sort by item pairs lexicographically
all_pairs = sorted(all_pairs, key=lambda x: (-1*int(x[2]), (x[0], x[1])))
# print the top 50 results without ties
result_printed = 0
current_support = None
for data in all_pairs:
    if result_printed >= 50: break
    item1 = data[0]
    item2 = data[1]
    support_count = data[2]
    support_frq = data[3]
    if support_count == current_support:
        continue
    print "%s\t%s\t%s\t%s" % (item1, item2, support_count, support_frq)
    current_support = support_count
    result_printed += 1

DAI62779	ELE17451	1592	0.051188
FRO40251	SNA80324	1412	0.045400
DAI75645	FRO40251	1254	0.040320
FRO40251	GRO85051	1213	0.039002
DAI62779	GRO73461	1139	0.036623
DAI75645	SNA80324	1130	0.036333
DAI62779	FRO40251	1070	0.034404
DAI62779	SNA80324	923	0.029678
DAI62779	DAI85309	918	0.029517
ELE32164	GRO59710	911	0.029292
DAI62779	DAI75645	882	0.028359
DAI62779	ELE92920	877	0.028198
FRO40251	FRO92469	835	0.026848
DAI62779	ELE32164	832	0.026752
DAI75645	GRO73461	712	0.022893
DAI43223	ELE32164	711	0.022861
DAI62779	GRO30386	709	0.022797
ELE17451	FRO40251	697	0.022411
DAI85309	ELE99737	659	0.021189
DAI62779	ELE26917	650	0.020900
GRO21487	GRO73461	631	0.020289
DAI62779	SNA45677	604	0.019421
ELE17451	SNA80324	597	0.019196
DAI62779	GRO71621	595	0.019131
DAI62779	SNA55762	593	0.019067
DAI62779	DAI83733	586	0.018842
ELE17451	GRO73461	580	0.018649
GRO73461	SNA80324	562	0.018070
DAI62779	GRO59710	561	0.018038
DAI62779	FRO80039	550	0.017684
DAI75645	ELE17451	547	0.017588
DAI62779	SNA93860	537	0.017266
D

<b>Discussion on H3.4</b>
<p>
The process took about 2 minutes and 18 seconds to run on a single AWS Linux Medium instance (and therefore network transfer is between mapper nodes is probably not an issue in this experiment).  The setup contains one mapper phase and one reducer phase.  Hadoop called mapper twice and reducer once.
</p>

* <b>HW3.5: Stripes</b>
<p>Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.

Report  the compute times for stripes job versus the Pairs job. 
Describe the computational setup used 
(E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called 
using Counters and report these counts. Discuss the differences in these counts between 
the Pairs and Stripes jobs</p>

In [27]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# record mapper invokation
sys.stderr.write("reporter:counter:custom_counters,mapper_count,1\n")
co_occurence = {}
total_baskets = 0
for line in sys.stdin:
    line = line.strip().upper()
    if line == '': continue
    total_baskets += 1
    lineArr = sorted(line.split(' '))
    for i, item1 in enumerate(lineArr):
        j = i + 1
        while j < len(lineArr):
            item2 = lineArr[j]
            if item1 not in co_occurence:
                co_occurence[item1] = {item2: 1}
            else:
                item1_co_occurence = co_occurence[item1]
                if item2 in item1_co_occurence:
                    item1_co_occurence[item2] += 1
                else:
                    item1_co_occurence[item2] = 1
            j += 1
print "%s\t%s\t%s" % ("stripe_strategy", "*", total_baskets)
for item1 in co_occurence:
    item1_co_occurence = co_occurence[item1]
    associative_output = '|'.join([str(k[0]) + ',' + str(k[1]) for k in item1_co_occurence.items()])
    print "%s\t%s\t%s" % ("stripe_strategy", item1, associative_output)

Overwriting mapper.py


In [30]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# record reducer invokation
sys.stderr.write("reporter:counter:custom_counters,reducer_count,1\n")

support_threshold = 100
total_baskets = 0
current_item = None
current_associated_counts = {}

for line in sys.stdin:
    line = line.strip()
    if line == '': continue
    lineArr = line.split('\t')
    item1 = lineArr[1]
    associated_info = lineArr[2]
    # gather total basket counts
    if item1 == "*":
        total_baskets += int(associated_info)
        continue
    if current_item == item1:
        # parse the associated array info
        for item2_info in associated_info.split('|'):
            item2 = item2_info.split(',')[0]
            item2_count = int(item2_info.split(',')[1])
            if item2 in current_associated_counts:
                current_associated_counts[item2] += item2_count
            else:
                current_associated_counts[item2] = item2_count
    else:
        # print out the current data
        if current_item != None:
            for item2 in current_associated_counts:
                if current_associated_counts[item2] > support_threshold:
                    print "%s\t%s\t%s\t%.6f" % (current_item, item2, current_associated_counts[item2], float(current_associated_counts[item2])/float(total_baskets))
        # parse new associative array info
        current_item = item1
        current_associated_counts = {}
        for item2_info in associated_info.split('|'):
            item2 = item2_info.split(',')[0]
            item2_count = int(item2_info.split(',')[1])
            if item2 in current_associated_counts:
                current_associated_counts[item2] += item2_count
            else:
                current_associated_counts[item2] = item2_count
# print the last record
if current_item != None:
    for item2 in current_associated_counts:
        if current_associated_counts[item2] > support_threshold:
            print "%s\t%s\t%s\t%.6f" % (current_item, item2, current_associated_counts[item2], float(current_associated_counts[item2])/float(total_baskets))

Overwriting reducer.py


In [31]:
!chmod +x mapper.py; chmod +x reducer.py;
!hdfs dfs -rm -r /w261/hw3/hw3_5_output;
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/lib/hadoop-*streaming*.jar \
-D stream.num.map.output.key.fields=2 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k2" \
-mapper mapper.py -file /data/w261/hw3/mapper.py \
-reducer reducer.py -file /data/w261/hw3/reducer.py \
-input /w261/hw3/ProductPurchaseData.txt \
-output /w261/hw3/hw3_5_output;

rm: `/w261/hw3/hw3_5_output': No such file or directory
16/01/31 02:29:45 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/data/w261/hw3/mapper.py, /data/w261/hw3/reducer.py] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob9173898191930945882.jar tmpDir=null
16/01/31 02:29:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 02:29:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 02:29:53 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 02:29:53 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 02:29:53 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/01/31 02:29:53 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/01/31 02:29:54 INFO mapreduce.J

In [33]:
# top 50 most co-occuring pairs.  break tie by taking the first in lexicographically increasing order
import subprocess
cat = subprocess.Popen(["hadoop", "fs", "-cat", "/w261/hw3/hw3_5_output/part*"], stdout=subprocess.PIPE)
all_pairs = []
for line in cat.stdout:
    all_pairs.append(line.strip().split('\t'))
# first sort by support, then sort by item pairs lexicographically
all_pairs = sorted(all_pairs, key=lambda x: (-1*int(x[2]), (x[0], x[1])))
# print the top 50 results without ties
result_printed = 0
current_support = None
for data in all_pairs:
    if result_printed >= 50: break
    item1 = data[0]
    item2 = data[1]
    support_count = data[2]
    support_frq = data[3]
    if support_count == current_support:
        continue
    print "%s\t%s\t%s\t%s" % (item1, item2, support_count, support_frq)
    current_support = support_count
    result_printed += 1

DAI62779	ELE17451	1592	0.051188
FRO40251	SNA80324	1412	0.045400
DAI75645	FRO40251	1254	0.040320
FRO40251	GRO85051	1213	0.039002
DAI62779	GRO73461	1139	0.036623
DAI75645	SNA80324	1130	0.036333
DAI62779	FRO40251	1070	0.034404
DAI62779	SNA80324	923	0.029678
DAI62779	DAI85309	918	0.029517
ELE32164	GRO59710	911	0.029292
DAI62779	DAI75645	882	0.028359
DAI62779	ELE92920	877	0.028198
FRO40251	FRO92469	835	0.026848
DAI62779	ELE32164	832	0.026752
DAI75645	GRO73461	712	0.022893
DAI43223	ELE32164	711	0.022861
DAI62779	GRO30386	709	0.022797
ELE17451	FRO40251	697	0.022411
DAI85309	ELE99737	659	0.021189
DAI62779	ELE26917	650	0.020900
GRO21487	GRO73461	631	0.020289
DAI62779	SNA45677	604	0.019421
ELE17451	SNA80324	597	0.019196
DAI62779	GRO71621	595	0.019131
DAI62779	SNA55762	593	0.019067
DAI62779	DAI83733	586	0.018842
ELE17451	GRO73461	580	0.018649
GRO73461	SNA80324	562	0.018070
DAI62779	GRO59710	561	0.018038
DAI62779	FRO80039	550	0.017684
DAI75645	ELE17451	547	0.017588
DAI62779	SNA93860	537	0.017266
D

<b>Discussion on HW3.5</b>
<p>
On the same hardware setup as in HW3.4, it took only 12 seconds to complete the process, which is significantly faster than the pair approach used in HW3.4.  The improvement should be coming from the reduction in the data needed to be transfered and sorted from mapper to reducer.  The setup contains one mapper phase and one reducer phase.  Hadoop called mapper twice and reducer once.
</p>